<a href="https://colab.research.google.com/github/JamainePillay/COMP703_AI/blob/main/Comp720AssignmentTwo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Jamaine Andrew Pillay
#Student Number: 222019095
!pip install tensorflow


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping


# 1.Data Preprocessing
file_path = '/content/sentiment_dataset.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1', on_bad_lines='skip', engine='python', header=None)
df = df[[0, 5]]
df.columns = ['sentiment', 'text']

df = df[df['sentiment'].isin([0, 4])]
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})

# Use only 10% of data
df = df.sample(frac=0.1, random_state=42)

# Train-test split
X = df['text'].values
y = df['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization and padding
vocab_size = 10000
max_length = 100
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_length)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_length)

# 2.Model Architectures and Training
def build_model(model_type='LSTM'):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=64))
    if model_type == 'LSTM':
        model.add(LSTM(64))
    elif model_type == 'GRU':
        model.add(GRU(64))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train LSTM
print("\nTraining LSTM")
lstm_model = build_model('LSTM')
lstm_model.fit(X_train_seq, y_train, validation_split=0.2, epochs=5, batch_size=128, callbacks=[early_stop], verbose=2)

# Evaluate LSTM
y_pred_lstm = (lstm_model.predict(X_test_seq) > 0.5).astype('int32')
print("\nLSTM Evaluation Report:\n")
print(classification_report(y_test, y_pred_lstm, labels=[0, 1], target_names=['Negative', 'Positive'], zero_division=0))

# Train GRU
print("\nTraining GRU")
gru_model = build_model('GRU')
gru_model.fit(X_train_seq, y_train, validation_split=0.2, epochs=5, batch_size=128, callbacks=[early_stop], verbose=2)

# Evaluate GRU
y_pred_gru = (gru_model.predict(X_test_seq) > 0.5).astype('int32')
print("\nGRU Evaluation Report:\n")
print(classification_report(y_test, y_pred_gru, labels=[0, 1], target_names=['Negative', 'Positive'], zero_division=0))



Training LSTM
Epoch 1/5
800/800 - 152s - 191ms/step - accuracy: 0.7634 - loss: 0.4877 - val_accuracy: 0.7909 - val_loss: 0.4486
Epoch 2/5
800/800 - 145s - 182ms/step - accuracy: 0.8125 - loss: 0.4104 - val_accuracy: 0.7904 - val_loss: 0.4409
Epoch 3/5
800/800 - 203s - 253ms/step - accuracy: 0.8300 - loss: 0.3740 - val_accuracy: 0.7925 - val_loss: 0.4514
Epoch 4/5
800/800 - 193s - 241ms/step - accuracy: 0.8451 - loss: 0.3415 - val_accuracy: 0.7863 - val_loss: 0.4722
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step

LSTM Evaluation Report:

              precision    recall  f1-score   support

    Negative       0.79      0.80      0.79     15878
    Positive       0.80      0.79      0.79     16122

    accuracy                           0.79     32000
   macro avg       0.79      0.79      0.79     32000
weighted avg       0.79      0.79      0.79     32000


Training GRU
Epoch 1/5
800/800 - 178s - 222ms/step - accuracy: 0.7627 - loss: 0.4869 - val_accuracy: 0.7894 - val_loss: 0.4484
Epo